In [1]:
from __future__ import division
from code.organize import *
from code.roughEDA import *
from code.survey_processor import *

import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
sv = survey('data/survey/')

In [3]:
sv.processor()

In [4]:
sv.fin_data.shape

(4504, 290)

In [5]:
sv.data.shape

(4504, 55)

In [7]:
import sklearn

In [11]:
sklearn.__version__

'0.16.1'

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
 
from sklearn.datasets import load_boston
boston = load_boston()

In [19]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
iris = load_iris()

In [20]:
rf = RandomForestClassifier(max_depth = 4)
idx = range(len(iris.target))
np.random.shuffle(idx)
 
rf.fit(iris.data[idx][:100], iris.target[idx][:100])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [25]:
instance = iris.data[idx][100:101]
print rf.predict(instance)

[0]


In [ ]:
simple_bar(df.marital)

In [ ]:
stack_bar(df.happy,df.teens)

In [ ]:
temp.plot(kind = 'pie',subplots=True,figsize=(20,8),autopct='%.2f');

#plt.legend(loc = (0.5, 0), ncol=5)

In [ ]:
h_bar(df.wrkstat,df.happy)

In [ ]:
temp.plot(kind = 'barh',figsize=(8,8))

In [ ]:
simple_bar(df.wrkstat, size = (20,10))

In [ ]:
pie_chart(df.wrkstat,df.marital,(20,20))

In [ ]:
h_bar(df.wrkstat,df.marital,(20,20))